In [1]:
import torch

from transformers import pipeline

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
classifier = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english", device=device)

text = "I'm proud of myself."
result = classifier(text)[0]

print("Input Text:", text)
print(f"The predicted tone is '{result['label']}' with a confidence score of {result['score']:.4f}.")


Device set to use cuda


Input Text: I'm proud of myself.
The predicted tone is 'POSITIVE' with a confidence score of 0.9999.


In [3]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=device)

text = "The prime minister announced a new climate policy to reduce carbon emissions."
candidate_labels = ["politics", "environment", "sports", "entertainment", "technology"]

result = classifier(text, candidate_labels)
    
    
print("Input Text:", text)
print("\nTop Predictions:")
for label, score in zip(result["labels"], result["scores"]):
    print(f"{label:15} → {score:.4f}")

Device set to use cuda


Input Text: The prime minister announced a new climate policy to reduce carbon emissions.

Top Predictions:
environment     → 0.6812
politics        → 0.2844
technology      → 0.0193
sports          → 0.0081
entertainment   → 0.0071


In [4]:
generator = pipeline("text-generation", model="gpt2", device=device)

prompt = "In a world ruled by machines, a scientist discovered a hidden code that could"
output = generator(prompt,
                    max_new_tokens=100, 
                    num_return_sequences=2,
                    top_k=50, # keep only the top k highest-probability ones
                    top_p=0.95, # nucleus sampling: he smallest possible set of tokens whose cumulative probability ≥ p
                    temperature=0.8, # Controls randomness
                    repetition_penalty=1.2)

for idx in range(len(output)):
    print(output[idx]['generated_text'])

Device set to use cuda
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a world ruled by machines, a scientist discovered a hidden code that could make your life more difficult if you don't get enough sleep.
/Getty Images 2 / Getty 3 AP 4 Reuters 5 EPA 6 / REUTERS 7/50 14 September 2018 Speaking in Malmo today he announced his support for an anti-fox hunting policy which would prevent the deployment of US military personnel to combat or intervene overseas, during another event at National Farmers Park on August 18th this year PA wire 8"soDeliveryDate 17 Indian Prime Minister Narendra Modi greets schoolchildren after him said there was "no
In a world ruled by machines, a scientist discovered a hidden code that could unlock one of the most powerful weapons in human history.
 "The program is called D-Gravity," said Dr. Andrew Hennig from Harvard Medical School who was not involved with the study or its findings at UMass Lowell University's Center for Computational Biology and Biotechnology but acknowledged it may be just what our ancestors did when they we

In [5]:
unmasker = pipeline("fill-mask", model="bert-base-uncased")
result = unmasker("The capital of France is [MASK].",
                  top_k = 5)

for prediction in result:
    print(f"{prediction['sequence']} (score: {prediction['score']:.4f})")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


the capital of france is paris. (score: 0.4168)
the capital of france is lille. (score: 0.0714)
the capital of france is lyon. (score: 0.0634)
the capital of france is marseille. (score: 0.0444)
the capital of france is tours. (score: 0.0303)


In [6]:
ner = pipeline("ner", model="dslim/bert-base-NER", aggregation_strategy="simple")
results = ner("My friend Sarah flew to the United Arab Emirates with Emirates Airlines.")

for result in results:
    print(f"{result['entity_group']}: {result['word']} (score: {result['score']:.4f})")

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


PER: Sarah (score: 0.9995)
LOC: United Arab Emirates (score: 0.9994)
ORG: Emirates Airlines (score: 0.9976)


In [7]:
question_answerer = pipeline("question-answering")

question = "When was the Eiffel Tower built?"
context = (
    "The Eiffel Tower is one of the most famous landmarks in the world. "
    "It was constructed in Paris, France, in 1889 as the entrance arch to the 1889 World's Fair. "
    "Standing 300 meters tall, it was the tallest man-made structure in the world for 41 years."
)
result = question_answerer(
    question=question,
    context=context,
)
print(f"Q: {question}")
print(f"A: {result['answer']} (score: {result['score']:.4f})")

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0


Q: When was the Eiffel Tower built?
A: 1889 (score: 0.9755)


In [8]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

text = (
    """
    Norwegian Wood is a poignant coming-of-age novel by Haruki Murakami that explores themes of love, loss, depression, and emotional maturation. The story is narrated by Toru Watanabe, a quiet and introspective college student in 1960s Tokyo, who is haunted by memories of his youth—especially his relationship with Naoko, the emotionally fragile girlfriend of his best friend Kizuki, who died by suicide. 
    As Toru and Naoko grow closer, her psychological instability deepens, and she moves to a secluded mental health retreat. Their relationship is marked by longing and emotional distance, burdened by past traumas. Meanwhile, Toru meets Midori, a vibrant and outspoken classmate who represents a stark contrast to Naoko. Midori is bold, alive, and emotionally open, offering Toru a potential path toward healing and connection.
    The novel follows Toru's emotional journey as he is pulled between the memories of his past and the possibilities of the present. Through richly detailed introspection and deeply personal encounters, Murakami crafts a meditation on youth, death, desire, and the painful beauty of human connection.
    In the end, Toru is left with a sense of ambiguity—neither fully healed nor broken—still searching for meaning in a world marked by impermanence. Norwegian Wood is not a story with tidy resolutions, but rather one that lingers with readers through its melancholic tone and emotional depth.
    """
)

summary = summarizer(text)
print(summary[0]['summary_text'])

Device set to use cuda:0


Norwegian Wood is a poignant coming-of-age novel by Haruki Murakami. The story is narrated by Toru Watanabe, a quiet and introspective college student in 1960s Tokyo. The novel follows Toru's emotional journey as he is pulled between the memories of his past and the possibilities of the present.


In [9]:
en_fr_translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-fr", device=device)
fr_en_translator = pipeline("translation", model="Helsinki-NLP/opus-mt-fr-en", device=device)

english_text = "She bought fresh bread from the bakery every morning."
print(f"Original Text: {english_text}")

fr_translation = en_fr_translator(english_text)[0]['translation_text']
print(f"French Translation: {fr_translation}")

en_reconstructed = fr_en_translator(fr_translation)[0]['translation_text']
print(f"English Reconstruction: {en_reconstructed}")

/home/khashayar/miniconda3/envs/nlp_may_25/lib/python3.11/site-packages/transformers/models/marian/tokenization_marian.py:177: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cuda
Device set to use cuda


Original Text: She bought fresh bread from the bakery every morning.
French Translation: Elle achetait du pain frais à la boulangerie tous les matins.
English Reconstruction: She bought fresh bread from the bakery every morning.


In [10]:
from PIL import Image
import requests
from io import BytesIO

url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/pipeline-cat-chonk.jpeg"
image = Image.open(BytesIO(requests.get(url).content))

image_classifier = pipeline("image-classification", model="google/vit-base-patch16-224")

result = image_classifier(image)

for label in result:
    print(f"{label['label']}: {label['score']:.4f}")

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.
Device set to use cuda:0


lynx, catamount: 0.4335
cougar, puma, catamount, mountain lion, painter, panther, Felis concolor: 0.0348
snow leopard, ounce, Panthera uncia: 0.0324
Egyptian cat: 0.0239
tiger cat: 0.0229


In [11]:
transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-large-v3")

url = "https://huggingface.co/datasets/Narsil/asr_dummy/resolve/main/mlk.flac"

result = transcriber(url)

print(result)

Device set to use cuda:0
/home/khashayar/miniconda3/envs/nlp_may_25/lib/python3.11/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


{'text': ' I have a dream that one day this nation will rise up and live out the true meaning of its creed.'}
